In [1]:
%matplotlib inline
from astropy.io import fits
from matplotlib import pyplot as plt
import numpy as np
from astropy.utils.data import get_pkg_data_filename

In [2]:
#start of bruno's code

import subprocess as sp

import glob

import astropy.io.fits as pyfits



def scampparams():

    pars = ["XWIN_IMAGE", "YWIN_IMAGE", "ERRAWIN_IMAGE", "ERRBWIN_IMAGE",\

            "ERRTHETAWIN_IMAGE", "FLUX_AUTO", "FLUXERR_AUTO", "FLAGS",\

            "FLAGS_WEIGHT", "FLUX_RADIUS", "MAG_AUTO","ALPHA_J2000","DELTA_J2000"]



    f = open("scamp.param","w")

    for p in pars:

        f.write(f"{p}\n")

    f.close()

    return None


In [3]:
scampparams()

# SANITY check of Zeropoint calculations
#### note that the below test example is just a weird one?  header keywords & values arent there for some reason... ...maybe b/c this was a "total" file?

In [4]:
#looking at hst fits data to get values needed for remaining zeropt calculations (my code)--(test1_image)

#verifying calculation matches known value from bruno
test1_file = fits.open('/data1/rowland/fits_data/hst_14096_i1_acs_wfc_total_drz.fits') #uses acs f435w
#test_file.info #shows theres 1 header, 4 images, 1 BinTable images - well [1] should be the SCI image then


test1_header0 = test1_file[0].header #header
''' 
test1_header1 = test1_file[1].header #science image 'SCI'
test1_header2 = test1_file[2].header #weights image 'WHT'
test1_header3 = test1_file[3].header #something we wont use image 'CTX'
test1_header4 = test1_file[4].header #wait where did this come from 'EXP'
test1_header5 = test1_file[5].header #whatever a bintable is 'HDRTAB' think it formats header list
''' 
#print(test1_header1['PHOTFLAM'])

#test1_file[0].header #goes from 0-5
#test1_header0["FILTER"] #NOT FOUND
test1_header0["FILTER2"] #returns 'F435W'


'F435W'

In [5]:
#trying another file (test2 image) b/c maybe its just that one? ------that one IS just weird....--test2 image
#verifying calculation matches known value from bruno
test2_file = fits.open('/data1/rowland/fits_data/hst_mos_1041271_acs_wfc_f814w_drz.fits') #uses acs f435w
#test_file.info #shows theres 1 header, 4 images, 1 BinTable images - well [1] should be the SCI image then

test2_file[0].header #goes from 0-4

test2_header0 = test2_file[0].header #header
test2_header1 = test2_file[1].header #science image 'SCI'
test2_header2 = test2_file[2].header #weights image 'WHT'
test2_header3 = test2_file[3].header #something we wont use image 'CTX'
test2_header4 = test2_file[4].header #wait where did this come from 'EXP'
 
#print('PHOTFLAM:',test2_header1['PHOTFLAM'])
#print('PHOTPLAM:',test2_header1['PHOTPLAM'])

PHOTFLAM_test2 = test2_header1['PHOTFLAM'] #7.0073061E-20 / inverse sensitivity, ergs/cm2/Ang/electron 
PHOTPLAM_test2 = test2_header1['PHOTPLAM'] #8.0593491E+03 / Pivot wavelength (Angstroms)
#test2_header0["FILTER"] #NOT FOUND
#print('FILTER2:',test2_header0["FILTER2"]) #returns 'F814W'
#print('FILTER1:',test2_header0['FILTER1']) #returns 'CLEAR1L'

In [6]:
#ZEROpoint calculation for test2

ZP_f814w = (-2.5 * np.log10(PHOTFLAM_test2)) - (5 * np.log10(PHOTPLAM_test2)) - 2.408
print(ZP_f814w, 'should match expected value of:', "f814w:25.947")
print('this is good!')

25.94662243630965 should match expected value of: f814w:25.947
this is good!


# NOTE TO SELF: 
### [for trying to automate selections, idk why but filter changes from 1 to 2 for correct name (see above test2 & below f850lp for an example)]

# Calculating ZEROpoints of remaining filters

In [7]:
#calculating remaining 3 zeropoints (f850lp, f775w, f625w)


#first f850lp
f850lp_file = fits.open('/data1/rowland/fits_data/hst_mos_1041271_acs_wfc_f850lp_drz.fits')
f850lp_header0 = f850lp_file[0].header #header

f850lp_header1 = f850lp_file[1].header #science image 'SCI'
f850lp_header2 = f850lp_file[2].header #weights image 'WHT'
f850lp_header3 = f850lp_file[3].header #something we wont use image 'CTX'
f850lp_header4 = f850lp_file[4].header #wait where did this come from 'EXP'
#f850lp_header5 = f850lp_file[5].header #whatever a bintable is 'HDRTAB' think it formats header list

#print(f850lp_header1['PHOTFLAM']) 
#f850lp_file.info
#f850lp_file[0].header #goes from 0-4
#f850lp_header0["FILTER"] #NOT FOUND
#print(f850lp_header0["FILTER2"]) #returns 'CLEAR2L'
print('FILTER1:',f850lp_header0["FILTER1"])#returns 'F850LP'

#needed values
PHOTFLAM_f850lp = f850lp_header1['PHOTFLAM']#1.5213061E-19 #/ inverse sensitivity, ergs/cm2/Ang/electron 
PHOTPLAM_f850lp = f850lp_header1['PHOTPLAM']#9.0336270E+03 #/ Pivot wavelength (Angstroms)   

FILTER1: F850LP


In [8]:
#second f775w

f775w_file = fits.open('/data1/rowland/fits_data/hst_mos_1041271_acs_wfc_f775w_drz.fits')

#f775w_file.info
#f775w_file[0].header #goes from 0-4

f775w_header0 = f775w_file[0].header #header
f775w_header1 = f775w_file[1].header #science image 'SCI'
f775w_header2 = f775w_file[2].header #weights image 'WHT'
f775w_header3 = f775w_file[3].header #something we wont use image 'CTX'
f775w_header4 = f775w_file[4].header #wait where did this come from 'EXP'

##print('PHOTFLAM:',f775w_header1['PHOTFLAM']) 
#f775w_header1

#print('FILTER:',f775w_header0["FILTER"]) #NOT FOUND
print('FILTER2:',f775w_header0["FILTER2"]) #returns 'CLEAR2L'
print('FILTER1:',f775w_header0["FILTER1"])#returns 'F775W'

#needed values
PHOTFLAM_f775w = f775w_header1['PHOTFLAM']#9.9942165E-20 / inverse sensitivity, ergs/cm2/Ang/electron
PHOTPLAM_f775w = f775w_header1['PHOTPLAM']#7.6936714E+03 / Pivot wavelength (Angstroms) 

FILTER2: CLEAR2L
FILTER1: F775W


In [9]:
#third f625w

f625w_file = fits.open('/data1/rowland/fits_data/hst_mos_1041271_acs_wfc_f625w_drz.fits')

#f625w_file.info
###f625w_file[4].header #goes from 0-4

f625w_header0 = f625w_file[0].header #header
f625w_header1 = f625w_file[1].header #science image 'SCI'
f625w_header2 = f625w_file[2].header #weights image 'WHT'
f625w_header3 = f625w_file[3].header #something we wont use image 'CTX'
f625w_header4 = f625w_file[4].header #wait where did this come from 'EXP'

print('PHOTFLAM:',f625w_header1['PHOTFLAM']) 
print('PHOTPLAM:',f625w_header1['PHOTPLAM'])
#f625w_header1

#print('FILTER:',f625w_header0["FILTER"]) #NOT FOUND
print('FILTER2:',f625w_header0["FILTER2"]) #returns 'CLEAR2L'
print('FILTER1:',f625w_header0["FILTER1"])#returns 'F625W'

#needed values
PHOTFLAM_f625w = f625w_header1['PHOTFLAM']#1.1931234E-19 / inverse sensitivity, ergs/cm2/Ang/electron 
PHOTPLAM_f625w = f625w_header1['PHOTPLAM']#6.3120513E+03 / Pivot wavelength (Angstroms) 

PHOTFLAM: 1.1931234e-19
PHOTPLAM: 6312.0513
FILTER2: CLEAR2L
FILTER1: F625W


In [10]:
photflam_list = [PHOTFLAM_f850lp, PHOTFLAM_f775w, PHOTFLAM_f625w]
photplam_list = [PHOTPLAM_f850lp, PHOTPLAM_f775w, PHOTPLAM_f625w]
#phot_list = [(PHOTFLAM_test2, PHOTPLAM_test2)] #tested for proof of concept - worked!
phot_list = [(PHOTFLAM_f850lp, PHOTPLAM_f850lp), (PHOTFLAM_f775w, PHOTPLAM_f775w), (PHOTFLAM_f625w, PHOTPLAM_f625w)]
ZPs = []
for x,y in phot_list:
    Zeropts = (-2.5 * np.log10(x)) - (5 * np.log10(y)) - 2.408
    ZPs.append(Zeropts)
ZPs = np.array(ZPs)
print('in the order of f850lp, f775w, f625w, the zeropts are:',ZPs)

in the order of f850lp, f775w, f625w, the zeropts are: [24.85714771 25.66195995 25.89943399]


# Making dictionaries with all filter values included

In [11]:
#creating needed dictionaries with calculated zeropoints included

#need a dictionary for zeropoint values of each filter
zeropoints = {"F435W":25.665,"F606W":26.493,"F814W":25.97,"F105W":26.2687,"F125W":26.2303,"F140W":24.4524,\
              "F160W":25.9463, "F625W":24.85714771, "F775W":25.66195995, "F850LP":25.89943399}

#need to create a dictionary for the 10 indiviual filters
filters_used = {'F435W','F606W','F814W','F625W','F775W','F850LP','F105W','F125W','F140W','F160W'}

# SANITY  check of splitting 'sci' & 'wht' frames

In [12]:
#testing what values for loop gathers --test3 image

test3_image = fits.open('/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz.fits') # equal to 'hdu' in for loop

test3_header = test3_image[0].header 
test3_filename = test3_header["FILTER"] #so this will work for SE files, prob not NW ones tho
print('test3_filename:',test3_filename) #seeing if this worked
test3_gain = float(test3_image[0].header["CCDGAIN"])
print('test3_gain:',test3_gain) #not sure what this means....
test3_exptime = float(test3_image[0].header['EXPTIME'])
print('test3_exptime:',test3_exptime) #exposure time
#other stuff in code
test3_headerOut = test3_image["SCI"].header
#this part builds a header from available info if filter name doesnt match list
    #print(test_headerOut) #prints the science frame header for each file
    #lines after that pull values from frame 0 to add to science frame header
    #outname uses science frame location as name of 
test3_dataOUt = test3_image["SCI"].data/test3_gain
#print(test_dataOUt) #wtf is this?
#for i in test_dataOUt: 
#    print(i)
print('shape of test3_dataOUt:',np.shape(test3_dataOUt)) #(2069, 2124)
print('shape of test3_image.data:',np.shape(test3_image["SCI"].data)) #(2069,2124) (#arrays,len of each array) #all the data is 0 matrices tho
test3_image['SCI'].data[0]

test3_filename: F160W
test3_gain: 2.5
test3_exptime: 1005.872924
shape of test3_dataOUt: (2069, 2124)
shape of test3_image.data: (2069, 2124)


array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [13]:
#figured out error - below code splits & rewrites files into original & sci & sci.wht files--test4 image 

#why getting no "CCDGAIN" errors now
#somehow (overwrite below?) some filenames changed to "_sci.wht.fits" to end of filenames
test4_SEfile = fits.open('/data1/rowland/fits_data/fits_data_Doubles_SE/hst_14096_j2_wfc3_ir_f125w_drz_SE.fits')
#test_SEfile[0].header
test4_SEfile.info

<bound method HDUList.info of [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7f73cba77128>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7f73cba8e908>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7f73cba93eb8>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7f73cbaa0a58>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7f73cba5c748>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7f73cba67ac8>]>

In [14]:
#example header to see if keywords 'FILTER', 'FILTER1', 'FILTER2' exist
example = fits.open('/data1/rowland/fits_data/fits_data_Doubles_SE/hst_14096_j2_wfc3_ir_f105w_drz_SE.fits')
example[0].header


SIMPLE  =                    T / Fits standard                                  
BITPIX  =                   16 / Bits per pixel                                 
NAXIS   =                    0 / Number of axes                                 
EXTEND  =                    T / File may contain extensions                    
ORIGIN  = 'NOAO-IRAF FITS Image Kernel July 2003' / FITS file originator        
DATE    = '2017-09-01         ' / Date FITS file was generated                  
IRAF-TLM= '2017-09-01T21:09:17' / Time of last modification                     
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
NEXTEND =                    4 / Number of standard extensions                  
FILENAME= 'hst_14096_j2_wfc3_ir_f105w_run_1_drz.fits' / name of file            
FILETYPE= 'SCI      '          / type of data found in data file                
                            

# Splitting 'sci' & 'wht' files

In [15]:
# need to split 3 files from hst fits data files for sextractor intake reasons
# CAN ONLY RUN THIS ONCE B/C IT REWRITES FILES AFTER SEPARATING! will get error if done more than once
filterImages = {}
#only doing on doubles/SE files first to test on smaller dataset and its in a different location anyway
all_SE_files = glob.glob('/data1/rowland/all_fits_data/hst*.fits')

for image in all_SE_files:
    #if image == "%s_SE"%(image[:-5]): #"%sdrz"%(image[:-5]): #use this part for none SE files
        print('this is image',image)

        hdu = pyfits.open(image)

        header = hdu[0].header
        
        try:
            if header["FILTER"] in filters_used: filterName = header["FILTER"] 
        except KeyError:
                pass
        try: 
             if header["FILTER1"] in filters_used: filterName = header["FILTER1"]
        except KeyError: 
                    pass
        try:
            if header["FILTER2"] in filters_used: filterName = header["FILTER2"]
        except KeyError:
                pass
        else:
            print('error determining filter used')
            
        gain = 2#float(hdu[0].header["CCDGAIN"])

        exptime = float(hdu[0].header["EXPTIME"])



        if filterName not in filterImages.keys():

            filterImages[filterName] = []



        print("%s_sci.fits"%(image[:-5])) #thats the output below of sci file locations

        headerOut = hdu["SCI"].header

        headerOut["FILTER"] = filterName
        headerOut["RADESYS"] = 'ICRS'
        headerOut["EXPTIME"] = exptime

        headerOut["GAIN"] = gain



        name = "/data1/rowland/all_fits_data/sci_n_wht_files/"

        outname = name + image.split("/")[-1][:-5]+"_sci.fits" 

        dataOUt = hdu["SCI"].data/gain
        

        pyfits.writeto(outname,dataOUt,header=headerOut,overwrite=True) #b/c overwrite cant repeat this cell...

        pyfits.writeto(name + image.split("/")[-1][:-5]+"_sci.wht.fits",hdu["WHT"].data,header=hdu["WHT"].header,overwrite=True)



        filterImages[filterName].append(outname)

        # break

print('FILTER IMAGES printout:',filterImages)

this is image /data1/rowland/all_fits_data/hst_14096_i2_wfc3_ir_f160w_drz.fits
/data1/rowland/all_fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.fits
this is image /data1/rowland/all_fits_data/hst_14096_j2_wfc3_ir_f105w_drz_SE.fits
/data1/rowland/all_fits_data/hst_14096_j2_wfc3_ir_f105w_drz_SE_sci.fits
this is image /data1/rowland/all_fits_data/hst_14096_i2_wfc3_ir_f140w_drz.fits
/data1/rowland/all_fits_data/hst_14096_i2_wfc3_ir_f140w_drz_sci.fits
this is image /data1/rowland/all_fits_data/hst_14096_j2_wfc3_ir_f140w_drz_SE.fits
/data1/rowland/all_fits_data/hst_14096_j2_wfc3_ir_f140w_drz_SE_sci.fits
this is image /data1/rowland/all_fits_data/hst_14096_i4_wfc3_ir_f105w_drz.fits
/data1/rowland/all_fits_data/hst_14096_i4_wfc3_ir_f105w_drz_sci.fits
this is image /data1/rowland/all_fits_data/hst_14096_j4_wfc3_ir_f160w_drz_SE.fits
/data1/rowland/all_fits_data/hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.fits
this is image /data1/rowland/all_fits_data/hst_14096_i4_wfc3_ir_f160w_drz.fits
/data1/rowland/

In [16]:
print('FILTER IMAGES printout:',filterImages.keys())

FILTER IMAGES printout: dict_keys(['F160W', 'F105W', 'F140W', 'F606W', 'F125W', 'F814W', 'F775W', 'F625W', 'F435W', 'F850LP'])


# SANITY check SExtractor start

In [17]:
### test 3 START
#error was test3_filterImages needs to begin as an empty dict before this for loop populates it

#testing below SExtractor run by first getting single file data in similar form---test3 image reused

single_test3_file = glob.glob('/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz.fits') #single file to test on
test3_filterImages = {} #{'f160w':'F160W'} #short dictionay of filters for a single file to test on


for image in single_test3_file:
    #gathers header info we want

    hdu3 = pyfits.open(image)

    header3 = hdu3[0].header
    #if 
    filterName3 = header3["FILTER"] #not sure this will work given above "NOTE TO SELF": fix = conditional if stmts

    gain3 = float(hdu3[0].header["CCDGAIN"])

    exptime3 = float(hdu3[0].header["EXPTIME"])


    #saves header info we want to each file (sci, wht) after separating out from original compressed file 
    if filterName3 not in test3_filterImages.keys():

        test3_filterImages[filterName3] = []



    print("%s_sci.fits"%(image[:-5])) #thats the output below of sci file locations

    headerOut3 = hdu3["SCI"].header

    headerOut3["FILTER"] = filterName3

    headerOut3["EXPTIME"] = exptime3

    headerOut3["GAIN"] = gain3



    outname3 = "%s_sci.TEST3.fits"%(image[:-5])

    dataOUt3 = hdu3["SCI"].data/gain3

    pyfits.writeto(outname3,dataOUt3,header=headerOut3,overwrite=True) #b/c overwrite cant repeat this cell...

    pyfits.writeto("%s_sci.wht.TEST3.fits"%(image[:-5]),hdu3["WHT"].data,header=hdu3["WHT"].header,overwrite=True)



    test3_filterImages[filterName3].append(outname3)

    # break

print('test3_FILTER_IMAGES printout:',test3_filterImages) #good prints out single file so i can test below w/o crashing cpu

/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.fits
test3_FILTER_IMAGES printout: {'F160W': ['/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits']}


In [18]:
#test3_filterImages[filterName3] #prints ['filename.fits']
test3_filterImages

{'F160W': ['/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits']}

In [19]:
#running test3 image outside of the for loop below so can compare anwsers

test3_thresh = 20.0
test3_sci = fits.open('/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz.fits')#_sci.TEST3.fits')

#simple entries based on gal.fit example with SEx

#these gives errors
#! sex -PARAMETERS_NAME default.param hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits -DETECT_THRESH {test3_thresh} #error opening file
#! sex -PARAMETERS_NAME default.param {test3_sci} -DETECT_THRESH {test3_thresh} #errors finding file

#this way works
! sex -PARAMETERS_NAME default.param /data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits -DETECT_THRESH {test3_thresh} #this one works!

> 
----- SExtractor 2.19.5 started on 2019-06-13 at 12:40:53 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights
> Initializing catalog
> Looking for hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits

> WARNING: Using ICRS instead of unknown astrometric reference frame: OTHER

----- Measuring from: hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits
      "Unnamed" / no ext. header / 2124x2069 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at lin

In [20]:
#trying above as a cycle now so for loop will work

test3_sci_cycle = ('/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits')
                  #/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3
! sex -PARAMETERS_NAME default.param {test3_sci_cycle} -DETECT_THRESH {test3_thresh} #works!

> 
----- SExtractor 2.19.5 started on 2019-06-13 at 12:40:54 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights
> Initializing catalog
> Looking for hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits

> WARNING: Using ICRS instead of unknown astrometric reference frame: OTHER

----- Measuring from: hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits
      "Unnamed" / no ext. header / 2124x2069 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at lin

      Objects: detected 297      / sextracted 293             

> Closing files
> 
> All done (in 0.2 s: 10013.4 lines/s , 1418.0 detections/s)


In [21]:
#trying above again but trying to save outputs

test3_SEoutput = ! sex -PARAMETERS_NAME default.param {test3_sci_cycle} -DETECT_THRESH {test3_thresh} 
#print('this is my printout:', test3_SEoutput) #weird format of print out but it did work

In [22]:
#testing below on single file first --test3 image reused
test3_thresh = 20.0

test3_skycat ="SDSS-R9"

for filter in test3_filterImages.keys(): #keys come after: (name:keys)

    test3_catalogs = ""

    for image in test3_filterImages[filter]:
        #print('image:', image)#?? gave 5 iterations to print F160W separately...weird

        test3_root = (image[:-5]) #this is the "..._sci.TEST3" file that was written during above for loop, not sure why the [:-5] tho
        print('this is test3_root:',test3_root) #ERROR B/C DOESNT END WITH '.FITS'
        
        ####adding .fits to end so !sex will work......alternatively just use image!!! 
        ####test3_fn = "%s.fits"%(image[:-5])
        ####print('hope this works:',test3_fn)
        
        #not sure how this gets inputted, maybe add to ! sex -PARAMETERS_NAME....line?
        test3_catalogs += f" {test3_root}.cat" #possibly CATALOG_NAME parameter using but why 5 printouts?
        #print(test3_catalogs) #??? no clue, just prints 5 rows of '.cat', with increasing array lengths 
        
        ####! sex -PARAMETERS_NAME default.param {test3_fn} -DETECT_THRESH {test3_thresh} #this works now!
        ! sex -PARAMETERS_NAME default.param {image} -DETECT_THRESH {test3_thresh} #this works now!


this is test3_root: /data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3
> 
----- SExtractor 2.19.5 started on 2019-06-13 at 12:40:56 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights
> Initializing catalog
> Looking for hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits

> WARNING: Using ICRS instead of unknown astrometric reference frame: OTHER

----- Measuring from: hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits
      "Unnamed" / no ext. header / 2124x2069 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at l

      Objects: detected 297      / sextracted 293             

> Closing files
> 
> All done (in 0.2 s: 10147.7 lines/s , 1437.1 detections/s)


In [23]:
#exploring test3_filterImages to get a sense of why for loop does 5 iterations in below cell 


#print(test3_filterImages) #{'f160w': 'F160W', 'F160W': ['/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits']}

#for i in test3_filterImages:
#    print(i) #(prints)f160w (next row) F160W

#type(test3_filterImages) #dict

#np.shape(test3_filterImages) #()

#len(test3_filterImages) #2

#test3_filterImages.header #dict so doesnt have, cant be a fits anymore

#test3_filterImages??

for image in test3_filterImages[filter]:
    print(image) #only single file pathway prints
    #print(image[:-5]) #file pathway w/o .fits (aka the last5 indices) obviously

/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits


# SExtractor start

In [29]:
# 2nd part of Bruno's code that actually runs sextractor AMENDED by me to run in jupyter nb

thresh = 50.0

skycat = "USNO-B1"#"SDSS-R9"#"2MASS"#
#filterImages comes from way above for loop of grouped files per filter

for filter in filterImages.keys(): #keys come after: (name:keys)

    catalogs = ""

    for image in filterImages[filter]:
        print('danielles images:',image)
        root = image[:-5]

        catalogs += f" {root}.cat"

        ! sex  {image} -DETECT_THRESH {thresh} -CATALOG_TYPE FITS_LDAC -CATALOG_NAME {root}.cat -CHECKIMAGE_TYPE NONE -PARAMETERS_NAME scamp.param -FILTER Y -FILTER_NAME gauss_1.5_3x3.conv -SEEING_FWHM 0.5 -PIXEL_SCALE 0.12 -MAG_ZEROPOINT {zeropoints[filter]} -ANALYSIS_THRESH {thresh} -DETECT_THRESH {thresh}
       # print(test_commandSEx)
        #! sex -PARAMETERS_NAME default.param {image} -DETECT_THRESH {thresh} #this works now!
        #break 
    #break
        #sp.call(commandSEx,shell=True)
  # # ! scamp {catalogs} -ASTREF_CATALOG {skycat}

    #print(commandSCAMP)
    all_images = " ".join(filterImages[filter])
    print('danielles test:',len(all_images))
    #sp.call(commandSCAMP,shell=True)
    #! swarp -IMAGEOUT_NAME elgordo_{filter}.fits -COMBINE_TYPE SUM -PIXELSCALE_TYPE MANUAL -PIXEL_SCALE 0.06 -GAIN_KEYWORD GAIN -WEIGHT_TYPE MAP_WEIGHT -WEIGHT_SUFFIX .wht.fits {all_images}

    break


    ##print(command)

    ##sp.call(command,shell=True)

danielles images: /data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f160w_drz_sci.fits
> 
----- SExtractor 2.19.5 started on 2019-06-13 at 12:48:19 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for hst_14096_i2_wfc3_ir_f160w_drz_sci.fits

> WARNING: Significant inaccuracy likely to occur in projection


> WARNING: Significant inaccuracy likely to occur in projection

----- Measuring from: hst_14096_i2_wfc3_ir_f160w_drz_sci.fits
      "Unnamed" / EXT. HEADER / 2124x2069 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  5

      Objects: detected 175      / sextracted 172             

> Closing files
> 
> All done (in 0.2 s: 9680.3 lines/s , 804.7 detections/s)
danielles images: /data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.fits
> 
----- SExtractor 2.19.5 started on 2019-06-13 at 12:48:20 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.fits

> WARNING: Significant inaccuracy likely to occur in projection


> WARNING: Significant inaccuracy likely to occur in projection

----- Measuring from: hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.fits
      "Unnamed" / EXT. HEADER / 2124x2069 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up b

      Objects: detected 157      / sextracted 155             

> Closing files
> 
> All done (in 0.2 s: 10275.8 lines/s , 769.8 detections/s)
danielles images: /data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f160w_drz_sci.fits
> 
----- SExtractor 2.19.5 started on 2019-06-13 at 12:48:20 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for hst_14096_i4_wfc3_ir_f160w_drz_sci.fits

> WARNING: Significant inaccuracy likely to occur in projection


> WARNING: Significant inaccuracy likely to occur in projection

----- Measuring from: hst_14096_i4_wfc3_ir_f160w_drz_sci.fits
      "Unnamed" / EXT. HEADER / 2124x2069 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up backgroun

      Objects: detected 175      / sextracted 173             

> Closing files
> 
> All done (in 0.2 s: 10132.1 lines/s , 847.2 detections/s)
danielles images: /data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f160w_drz_SE_sci.fits
> 
----- SExtractor 2.19.5 started on 2019-06-13 at 12:48:20 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for hst_14096_j2_wfc3_ir_f160w_drz_SE_sci.fits

> WARNING: Significant inaccuracy likely to occur in projection


> WARNING: Significant inaccuracy likely to occur in projection

----- Measuring from: hst_14096_j2_wfc3_ir_f160w_drz_SE_sci.fits
      "Unnamed" / EXT. HEADER / 2123x2068 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up 

      Objects: detected 158      / sextracted 156             

> Closing files
> 
> All done (in 0.2 s: 10066.6 lines/s , 759.4 detections/s)
danielles test: 345


In [28]:
#print({root})
#print(np.shape(all_images)) #len 345
print(all_images)

/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f160w_drz_sci.fits /data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.fits /data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f160w_drz_sci.fits /data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f160w_drz_SE_sci.fits


In [30]:
print(filterImages)

{'F160W': ['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f160w_drz_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f160w_drz_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f160w_drz_SE_sci.fits'], 'F105W': ['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f105w_drz_SE_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f105w_drz_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f105w_drz_SE_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f105w_drz_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_total_drz_SE_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_12477_32_acs_wfc_total_drz_sci.fits'], 'F140W': ['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f140w

In [31]:
print(filterImages[filter])

['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f160w_drz_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f160w_drz_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f160w_drz_SE_sci.fits']


In [35]:
for filter in filterImages:
    print(filter)

F160W
F105W
F140W
F606W
F125W
F814W
F775W
F625W
F435W
F850LP


In [46]:
for image in filterImages:
    print(image)

F160W
F105W
F140W
F606W
F125W
F814W
F775W
F625W
F435W
F850LP


In [47]:
for image in filterImages[filter]:
    print(image)

/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f850lp_drz_sci.fits
/data1/rowland/all_fits_data/sci_n_wht_files/hst_12477_33_acs_wfc_total_drz_sci.fits
/data1/rowland/all_fits_data/sci_n_wht_files/hst_12477_31_acs_wfc_total_drz_sci.fits


In [71]:
acs_files = {'F606W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f606w_drz_sci.fits', \
              'F625W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f625w_drz_sci.fits', \
              'F775W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f775w_drz_sci.fits', \
              'F814W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f814w_drz_sci.fits', \
              'F850LP': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f850lp_drz_sci.fits'}



In [88]:
#dictionary key:value

for filter,image in acs_files.items(): #acs_files.values(): #finally this gives datapaths #acs_files.keys():#this prints filters#acs_files.items():#prints whole dict
    print(image)
    print(filter) #acs_files.get())
    print('starting next cycle')

/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f606w_drz_sci.fits
F606W
starting next cycle
/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f625w_drz_sci.fits
F625W
starting next cycle
/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f775w_drz_sci.fits
F775W
starting next cycle
/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f814w_drz_sci.fits
F814W
starting next cycle
/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f850lp_drz_sci.fits
F850LP
starting next cycle


# running SExtractor on ACS filters to generate stars for PSF

In [89]:
acs_files = {'F606W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f606w_drz_sci.fits', \
              'F625W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f625w_drz_sci.fits', \
              'F775W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f775w_drz_sci.fits', \
              'F814W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f814w_drz_sci.fits', \
              'F850LP': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f850lp_drz_sci.fits'}


thresh_test = 20.0

for filter,image in acs_files.items():
    #print('trying to prove acutally doing cycles on each filter:')
    #print('these are the images:',image)
    #if filter in acs_filters:
    print('processing this image',image)
    #filtername = image
     #   print('SHOULD BE DOING THIS FILTER:',filter)
        #note output .cat file will save to /home/rowland/leaps folder instead of with other data, fix location or manually move later to sci_n_wht_files
    ! sex {image} -DETECT_THRESH {thresh_test} -CATALOG_NAME testPSF_{filter}.cat -PARAMETERS_NAME default.param -PIXEL_SCALE 0.12 -MAG_ZEROPOINT {zeropoints[filter]} -ANALYSIS_THRESH {thresh_test} -DETECT_THRESH {thresh_test}
    #else:
    #    print('should have skipped this filter:',filter)
    #break

processing this image /data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f606w_drz_sci.fits
> 
----- SExtractor 2.19.5 started on 2019-06-13 at 14:22:52 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights
> Initializing catalog
> Looking for hst_mos_1041271_acs_wfc_f606w_drz_sci.fits
----- Measuring from: hst_mos_1041271_acs_wfc_f606w_drz_sci.fits
      "Unnamed" / no ext. header / 11700x12100 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Sett

> Setting up background map at line: 9856
> Setting up background map at line: 9920
> Setting up background map at line: 9984
> Setting up background map at line:10048
> Setting up background map at line:10112
> Setting up background map at line:10176
> Setting up background map at line:10240
> Setting up background map at line:10304
> Setting up background map at line:10368
> Setting up background map at line:10432
> Setting up background map at line:10496
> Setting up background map at line:10560
> Setting up background map at line:10624
> Setting up background map at line:10688
> Setting up background map at line:10752
> Setting up background map at line:10816
> Setting up background map at line:10880
> Setting up background map at line:10944
> Setting up background map at line:11008
> Setting up background map at line:11072
> Setting up background map at line:11136
> Setting up background map at line:11200
> Setting up background map at line:11264
> Setting up background map at lin

> Line: 2300  Objects:      103 detected /       25 sextracted
> Line: 2325  Objects:      106 detected /       25 sextracted
> Line: 2350  Objects:      109 detected /       25 sextracted
> Line: 2375  Objects:      110 detected /       26 sextracted
> Line: 2400  Objects:      111 detected /       27 sextracted
> Line: 2425  Objects:      111 detected /       29 sextracted
> Line: 2450  Objects:      112 detected /       31 sextracted
> Line: 2475  Objects:      115 detected /       32 sextracted
> Line: 2500  Objects:      117 detected /       34 sextracted
> Line: 2525  Objects:      119 detected /       34 sextracted
> Line: 2550  Objects:      121 detected /       35 sextracted
> Line: 2575  Objects:      123 detected /       37 sextracted
> Line: 2600  Objects:      125 detected /       40 sextracted
> Line: 2625  Objects:      128 detected /       42 sextracted
> Line: 2650  Objects:      129 detected /       45 sextracted
> Line: 2675  Objects:      133 detected /       47 sex

> Line: 5375  Objects:      442 detected /      282 sextracted
> Line: 5400  Objects:      444 detected /      283 sextracted
> Line: 5425  Objects:      449 detected /      284 sextracted
> Line: 5450  Objects:      458 detected /      290 sextracted
> Line: 5475  Objects:      465 detected /      295 sextracted
> Line: 5500  Objects:      468 detected /      295 sextracted
> Line: 5525  Objects:      469 detected /      301 sextracted
> Line: 5550  Objects:      470 detected /      303 sextracted
> Line: 5575  Objects:      472 detected /      304 sextracted
> Line: 5600  Objects:      476 detected /      305 sextracted
> Line: 5625  Objects:      480 detected /      308 sextracted
> Line: 5650  Objects:      482 detected /      310 sextracted
> Line: 5675  Objects:      483 detected /      315 sextracted
> Line: 5700  Objects:      487 detected /      317 sextracted
> Line: 5725  Objects:      489 detected /      319 sextracted
> Line: 5750  Objects:      491 detected /      322 sex

> Line: 8275  Objects:      803 detected /      628 sextracted
> Line: 8300  Objects:      808 detected /      630 sextracted
> Line: 8325  Objects:      811 detected /      632 sextracted
> Line: 8350  Objects:      812 detected /      636 sextracted
> Line: 8375  Objects:      815 detected /      639 sextracted
> Line: 8400  Objects:      816 detected /      643 sextracted
> Line: 8425  Objects:      819 detected /      647 sextracted
> Line: 8450  Objects:      821 detected /      649 sextracted
> Line: 8475  Objects:      823 detected /      649 sextracted
> Line: 8500  Objects:      827 detected /      656 sextracted
> Line: 8525  Objects:      831 detected /      662 sextracted
> Line: 8550  Objects:      833 detected /      666 sextracted
> Line: 8575  Objects:      835 detected /      673 sextracted
> Line: 8600  Objects:      836 detected /      676 sextracted
> Line: 8625  Objects:      836 detected /      676 sextracted
> Line: 8650  Objects:      839 detected /      678 sex

...

> Setting up background map at line: 4224
> Setting up background map at line: 4288
> Setting up background map at line: 4352
> Setting up background map at line: 4416
> Setting up background map at line: 4480
> Setting up background map at line: 4544
> Setting up background map at line: 4608
> Setting up background map at line: 4672
> Setting up background map at line: 4736
> Setting up background map at line: 4800
> Setting up background map at line: 4864
> Setting up background map at line: 4928
> Setting up background map at line: 4992
> Setting up background map at line: 5056
> Setting up background map at line: 5120
> Setting up background map at line: 5184
> Setting up background map at line: 5248
> Setting up background map at line: 5312
> Setting up background map at line: 5376
> Setting up background map at line: 5440
> Setting up background map at line: 5504
> Setting up background map at line: 5568
> Setting up background map at line: 5632
> Setting up background map at lin

> Line:  675  Objects:        5 detected /        0 sextracted
> Line:  700  Objects:        5 detected /        0 sextracted
> Line:  725  Objects:        5 detected /        0 sextracted
> Line:  750  Objects:        5 detected /        0 sextracted
> Line:  775  Objects:        5 detected /        0 sextracted
> Line:  800  Objects:        6 detected /        0 sextracted
> Line:  825  Objects:        7 detected /        0 sextracted
> Line:  850  Objects:        7 detected /        0 sextracted
> Line:  875  Objects:        7 detected /        0 sextracted
> Line:  900  Objects:        7 detected /        0 sextracted
> Line:  925  Objects:        7 detected /        0 sextracted
> Line:  950  Objects:        7 detected /        0 sextracted
> Line:  975  Objects:        7 detected /        0 sextracted
> Line: 1000  Objects:        7 detected /        0 sextracted
> Line: 1025  Objects:        7 detected /        0 sextracted
> Line: 1050  Objects:        7 detected /        0 sex

> Line: 3575  Objects:       98 detected /       37 sextracted
> Line: 3600  Objects:      101 detected /       37 sextracted
> Line: 3625  Objects:      102 detected /       38 sextracted
> Line: 3650  Objects:      104 detected /       40 sextracted
> Line: 3675  Objects:      107 detected /       41 sextracted
> Line: 3700  Objects:      108 detected /       42 sextracted
> Line: 3725  Objects:      111 detected /       43 sextracted
> Line: 3750  Objects:      114 detected /       43 sextracted
> Line: 3775  Objects:      114 detected /       44 sextracted
> Line: 3800  Objects:      115 detected /       45 sextracted
> Line: 3825  Objects:      119 detected /       46 sextracted
> Line: 3850  Objects:      123 detected /       48 sextracted
> Line: 3875  Objects:      124 detected /       48 sextracted
> Line: 3900  Objects:      127 detected /       48 sextracted
> Line: 3925  Objects:      132 detected /       50 sextracted
> Line: 3950  Objects:      134 detected /       50 sex

> Line: 6475  Objects:      334 detected /      253 sextracted
> Line: 6500  Objects:      339 detected /      254 sextracted
> Line: 6525  Objects:      340 detected /      254 sextracted
> Line: 6550  Objects:      340 detected /      255 sextracted
> Line: 6575  Objects:      341 detected /      257 sextracted
> Line: 6600  Objects:      343 detected /      259 sextracted
> Line: 6625  Objects:      348 detected /      261 sextracted
> Line: 6650  Objects:      352 detected /      264 sextracted
> Line: 6675  Objects:      356 detected /      265 sextracted
> Line: 6700  Objects:      358 detected /      267 sextracted
> Line: 6725  Objects:      362 detected /      269 sextracted
> Line: 6750  Objects:      363 detected /      271 sextracted
> Line: 6775  Objects:      367 detected /      271 sextracted
> Line: 6800  Objects:      367 detected /      275 sextracted
> Line: 6825  Objects:      367 detected /      276 sextracted
> Line: 6850  Objects:      371 detected /      276 sex

> Line: 9350  Objects:      541 detected /      492 sextracted
> Line: 9375  Objects:      542 detected /      492 sextracted
> Line: 9400  Objects:      542 detected /      494 sextracted
> Line: 9425  Objects:      543 detected /      495 sextracted
> Line: 9450  Objects:      543 detected /      495 sextracted
> Line: 9475  Objects:      543 detected /      496 sextracted
> Line: 9500  Objects:      543 detected /      497 sextracted
> Line: 9525  Objects:      543 detected /      499 sextracted
> Line: 9550  Objects:      543 detected /      499 sextracted
> Line: 9575  Objects:      543 detected /      499 sextracted
> Line: 9600  Objects:      544 detected /      499 sextracted
> Line: 9625  Objects:      544 detected /      501 sextracted
> Line: 9650  Objects:      545 detected /      502 sextracted
> Line: 9675  Objects:      545 detected /      505 sextracted
> Line: 9700  Objects:      546 detected /      505 sextracted
> Line: 9725  Objects:      546 detected /      505 sex